**Installing the needed libraries**

In [1]:
pip install pubmed-lookup

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install elsapy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install git+http://github.com/mfcovington/pubmed-lookup.git@develop

  Cloning http://github.com/mfcovington/pubmed-lookup.git (to revision develop) to /private/var/folders/4b/jg0k5m617x3bpy86jy6b49p40000gn/T/pip-req-build-baoal5ux
  Running command git clone -q http://github.com/mfcovington/pubmed-lookup.git /private/var/folders/4b/jg0k5m617x3bpy86jy6b49p40000gn/T/pip-req-build-baoal5ux
  Created wheel for pubmed-lookup: filename=pubmed_lookup-0.2.3-cp37-none-any.whl size=10662 sha256=451e81445cb9454e58a066e1358e8b68490f79c7c5c191b08d8eb5b17a489e50
  Stored in directory: /private/var/folders/4b/jg0k5m617x3bpy86jy6b49p40000gn/T/pip-ephem-wheel-cache-bntfifz4/wheels/6c/bb/4e/bad8766ea99b3e71827432463e4f7fda40e6cc206d521e1b20
Successfully built pubmed-lookup
Note: you may need to restart the kernel to use updated packages.


**Generating the dataframe**

In [1]:
from Bio import Entrez
from Bio import Medline
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
Result = []

Entrez.email = "anandas.georges@gmail.com"     # Always tell NCBI who you are
my_molecules=["coibanol", "bastimolide", "coibamide", "briarellin", "santacruzamate", "gallinamide A", "coibacins", "bacilosarcin A", "medusamide", \
              "credneramides", "seco-briarellinone", "pseudopterane diterpene", "[d-MeAla11]-epimer"]
diseases=["Buruli Ulcer", "buruli ulcer", "Cancer", "cancer", "Chagas Disease", "chagas disease", "Chromoblastomycosis", "chromoblastomycosis", \
          "Cysticercosis", "cysticercosis", "Dengue Fever", "dengue fever", "Dracunculiasis", "dracunculiasis", "Guinea Worm Disease", "guinea worm disease", \
          "Echinococcosis", "echinococcosis", "Fascioliasis", "fascioliasis", "African Trypanosomiasis", "african trypanosomiasis", \
          "African Sleeping Sickness", "african sleeping sickness", "Leishmaniasis", "leishmaniasis", "Leprosy", "leprosy", "Hansen’s Disease", \
          "Hansen’s Disease", "hansen’s disease", "Lymphatic Filariasis", "lymphatic filariasis", "malaria", \
          "Mycetoma", "mycetoma", "Onchocerciasis", "onchocerciasis", "Rabies", "rabies", "Schistosomiasis", "schistosomiasis", \
          "Soil-transmitted Helminths", "soil-transmitted selminths", "Ascaris", "ascaris", "Hookworm", "hookworm", "Whipworm", "whipworm", \
          "Trachoma", "trachoma", "Yaws", "yaws", "Disease", "disease"]

for molecule in my_molecules:
    print(molecule)
    handle = Entrez.esearch(db="PMC", term=molecule, sort="relevance", retmode="xml")
    record = Entrez.read(handle)
    idlist = record["IdList"]

    if (int(record["Count"])>0):
      
      for index in list(record["IdList"]):
          handle = Entrez.efetch(db="PMC", id=index, rettype="medline")
          records = Medline.parse(handle)
          records = list(records)
          
          #to iterate trough records and to extract the papers' info
          for record in records: 
              title=record.get("TI", "?")

              authors=record.get("AU", "?")
              authors =','.join([str(item) for item in authors ])
              authorsnum=len(record.get("FAU", "?"))

              abstract = record.get("AB", "?")

              textaffiliacion=record.get("AD", "?")

              #To look for all Panamanian authors based on affiliation
              listpanama = re.findall(r"Panam[a|á]", str(textaffiliacion))
              
              
              
              #To calculate % of Panamanian authors in each paper
              modlistpanama=(len(listpanama) / len(textaffiliacion))
              if (modlistpanama==1):
                  countpanama=len(textaffiliacion)
              else:
                  countpanama=len(listpanama) #Num of Panamanian authors

              #score=Num of Panamanian authors/total authors
              if (countpanama == 0.0):
                  score=0
              elif (countpanama > 0.0):
                  score=modlistpanama

              #to find papers related with the list of NTD (neglected tropical diseases) + cancer and other diseases
              dis=[]
              for i in diseases:
                if i in abstract:
                  dis.append(i)
              disfound =', '.join([str(item) for item in dis])

              #to generate data frame with all info
              Result.append([molecule,index,title,authors,authorsnum,countpanama,score,disfound,dis])

publication_data = pd.DataFrame(Result)
#print (len(publication_data))
publication_data.rename(columns={0:'MOLECULE',1:'PMC',2:'TITLE',3:'AUTHORS',4:'QUANTITY',5:'Affiliation',6:'COUNT PANAMA',7:'Score', 8:'Diseases',9:'Diseases Array'}, inplace=True)
print(publication_data)

publication_data.to_csv("resultados.csv")

coibanol
bastimolide
coibamide


HTTPError: HTTP Error 429: Too Many Requests

**Generating Authorship and Co-authorship in Published Articles**

In [ ]:
sns.set_style("white")

Resulta = [] 
autores=[]
instituciones=[]
institucionesall=[]
cont=0

publication_data = pd.read_csv("resultados.csv")

print(publication_data )

print(publication_data.groupby(by=["MOLECULE"]).count() )

AU = publication_data.AUTHORS.str.get_dummies(sep=',').sum().sort_values(ascending=False)
TOP = AU.reset_index()            
TOP.rename(columns={0:'Abundance'}, inplace=True)
print(TOP)
print(TOP.iloc[0:10])

auseries=pd.DataFrame(autores)
print(auseries.apply(pd.Series.value_counts))
newauseries=(auseries.apply(pd.Series.value_counts))
auseries.to_csv("autores.csv")
newauseries.to_csv("autoresvalues.csv")

for ii in publication_data['Affiliation'].index:
  linea=publication_data['Affiliation'][ii]

listpanama = re.findall(r"Panam[á|a]", str(linea) )
if listpanama:
  listindicasat = re.findall(r"Instituto de Investigaciones Cient[í|i]ficas Avanzados y Sevicios de Alta Tecnolog[í|i]a", str(linea))

if listindicasat:
  instituciones.append(listindicasat)
  
listindicasat2 = re.findall(r"INDICASAT", str(linea))
if listindicasat2:
  instituciones.append(listindicasat2)

listuniv = re.findall(r"University of Panam[á|a]", str(linea))

if listuniv:
  instituciones.append(listuniv)

listuniv2 = re.findall(r"Universidad de Panam[á|a]", str(linea))
if listuniv2:
  instituciones.append(listuniv2)

gorgas = re.findall(r"Gorgas", str(linea))
if gorgas:
  instituciones.append(gorgas)

tecnologica = re.findall(r"Universidad Tecnol[ó|o]gica de Panam[á|a]", str(linea))
if tecnologica:
  instituciones.append(tecnologica)    

Smithsonian= re.findall(r"Smithsonian Tropical Research Institute", str(linea))
if Smithsonian:
  instituciones.append(Smithsonian)

for i in instituciones:
  for x in i:
    institucionesall.append(x)
print(autores)

x=input()

print("total de Instituciones Panameñas = ",len(institucionesall))

instseries=pd.DataFrame(institucionesall)
print(instseries.apply(pd.Series.value_counts))
newintseries=(instseries.apply(pd.Series.value_counts))

instseries.to_csv("instituciones.csv")
newintseries.to_csv("institucionesvalues.csv")

print(publication_data.columns)
just_panama=publication_data[publication_data['COUNT PANAMA'] >= 1]
print(just_panama)
print(just_panama['COUNT PANAMA'])
print(just_panama['COUNT PANAMA']/just_panama['QUANTITY'])

score2=just_panama['COUNT PANAMA']/just_panama['QUANTITY']

plt.figure()
plt.hist(score2,bins=4)
plt.ylabel('Frequency')
plt.xlabel('Score')
plt.show()

plt.figure()
plt.hist(publication_data['COUNT PANAMA'],bins=5)
plt.ylabel('Frequency')
plt.xlabel('Counts')
plt.show()

FileNotFoundError: ignored

**Dataframe**

In [ ]:
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("white")
publication_data = pd.DataFrame(Result)
#print (len(publication_data))
publication_data.rename(columns={0:'MOLECULE',1:'PMC',2:'TITLE',3:'AUTHORS',4:'QUANTITY',5:'COUNT PANAMA',6:'Score', 7:'Diseases'}, inplace=True)
publication_data



,MOLECULE,PMC,TITLE,AUTHORS,QUANTITY,COUNT PANAMA,Score,Diseases,8
0,coibanol,4148033,"Coibanoles, a new class of meroterpenoids prod...","Molinar E,Rios N,Spadafora C,Arnold AE,Coley P...",8,7,0.875,,[]
1,coibanol,3475268,Meroterpenes from Endophytic Fungus A1 of Mang...,"Mei WL,Zheng B,Zhao YX,Zhong HM,Chen XLW,Zeng ...",10,0,0.000,,[]
2,bastimolide,7180531,Palstimolide A: A Complex Polyhydroxy Macrolid...,"Keller L,Siqueira-Neto JL,Souza JM,Eribez K,La...",7,0,0.000,malaria,[malaria]
3,bastimolide,5450532,A Review Study on Macrolides Isolated from Cya...,"Wang M,Zhang J,He S,Yan X",4,0,0.000,,[]
4,bastimolide,6896759,Antiplasmodial natural products: an update.,"Tajuddeen N,Van Heerden FR",2,0,0.000,malaria,[malaria]
...,...,...,...,...,...,...,...,...,...
147,pseudopterane diterpene,5852477,Chloro-Furanocembranolides from Leptogorgia sp...,"Gallardo AB,Díaz-Marrero AR,de la Rosa JM,D’Cr...",8,3,0.375,,[]
148,pseudopterane diterpene,3728638,Use of a palladium(II)-catalyzed oxidative kin...,"Meyer ME,Phillips JH,Ferreira EM,Stoltz BM",4,0,0.000,,[]
149,pseudopterane diterpene,3677362,New taxa and revisionary systematics of alcyon...,Williams GC,1,0,0.000,,[]
150,[d-MeAla11]-epimer,4342349,Synthesis and Biological Evaluation of the [d-...,"Nabika R,Suyama TL,Hau AM,Misu R,Ohno H,Ishmae...",9,0,0.000,cancer,[cancer]


**Calculating how many papers, on the topic, has each author**

In [ ]:
#Separate the names in the cells of Authors column.
AU = publication_data.AUTHORS.str.get_dummies(sep=',').sum().sort_values(ascending=False)
TOP = AU.reset_index()            
TOP.rename(columns={0:'Abundance'}, inplace=True)
TOP

,index,Abundance
0,Gerwick WH,24
1,McPhail KL,14
2,Gutiérrez M,10
3,Rodríguez AD,8
4,Ishmael JE,8
...,...,...
635,Tang N,1
636,Tan M,1
637,LaMonte GM,1
638,Lai PY,1


**Extracting diseases names from abstracts**

In [ ]:
molecules_diseases={}
enfermedades=[]
enfermedades_por_moleculas=[]

for i,k in enumerate(publication_data["MOLECULE"]):
  for m,n in enumerate(my_molecules):
    if k==n:
      enfermedades.append([k, publication_data[8][i]])
  enfermedades_por_moleculas.append(enfermedades)
print(len(enfermedades))

152


**Creating a dictionary molecule-diseases**

In [ ]:
import itertools
diseases_by_molecule={}

coibaI=[]
coibaII=[]
bastiI=[]
bastiII=[]
coibamiI=[]
coibamiII=[]
briarI=[]
briarII=[]
santaI=[]
santaII=[]
gallinaI=[]
gallinaII=[]
coibacI=[]
coibacII=[]
baciloI=[]
baciloII=[]
medusaI=[]
medusaII=[]
credneI=[]
credneII=[]
secoI=[]
secoII=[]
pseudoI=[]
pseudoII=[]
epimerI=[]
epimerII=[]

# chain.from_iterable(['ABC', 'DEF']) --> A B C D E F
def from_iterable(iterables):
  for it in iterables:
    for element in it:
      yield element

#papers about Coibanol related with diseases
coibaI.append(enfermedades[:2])
for i,j in enumerate(coibaI[0]):
  for k,l in enumerate(j[1:]):
    coibaII.append(l)
coibanol=from_iterable(coibaII)
x1=list(set(coibanol))
diseases_by_molecule['coibanol']=x1

#papers about Bastimolide related with diseases
bastiI.append(enfermedades[2:15])
for i,j in enumerate(bastiI[0]):
  for k,l in enumerate(j[1:]):
    bastiII.append(l)
bastimolide=list(from_iterable(bastiII))
x2=list(set(bastimolide))
diseases_by_molecule['bastimolide']=x2

#papers about Coibamide related with diseases
coibamiI.append(enfermedades[15:35])
for i,j in enumerate(coibamiI[0]):
  for k,l in enumerate(j[1:]):
    coibamiII.append(l)
coibamide=from_iterable(coibamiII)
x3=list(set(coibamide))
diseases_by_molecule['coibamide']=x3

#papers about Briarellin related with diseases
briarI.append(enfermedades[35:52])
for i,j in enumerate(briarI[0]):
  for k,l in enumerate(j[1:]):
    briarII.append(l)
briarellin=from_iterable(briarII)
x4=list(set(briarellin))
diseases_by_molecule['briarellin']=x4

#papers about Santacruzamate related with diseases
santaI.append(enfermedades[52:72])
for i,j in enumerate(santaI[0]):
  for k,l in enumerate(j[1:]):
    santaII.append(l)
santacruzamate=from_iterable(santaII)
x5=list(set(santacruzamate))
diseases_by_molecule['santacruzamate']=x5

#papers about Gallinamide A related with diseases
gallinaI.append(enfermedades[72:92])
for i,j in enumerate(gallinaI[0]):
  for k,l in enumerate(j[1:]):
    santaII.append(l)
gallinamide_A=from_iterable(gallinaII)
x6=list(set(gallinamide_A))
diseases_by_molecule['gallinamide_A']=x6

#papers about Coibacins related with diseases
coibacI.append(enfermedades[92:105])
for i,j in enumerate(coibacI[0]):
  for k,l in enumerate(j[1:]):
    coibacII.append(l)
coibacins=from_iterable(coibacII)
x7=list(set(coibacins))
diseases_by_molecule['coibacins']=x7

#papers about Bacilosarcin related with diseases
baciloI.append(enfermedades[105:115])
for i,j in enumerate(baciloI[0]):
  for k,l in enumerate(j[1:]):
    baciloII.append(l)
bacilosarcin=from_iterable(baciloII)
x8=list(set(bacilosarcin))
diseases_by_molecule['bacilosarcin']=x8

#papers about Medusamide related with diseases
medusaI.append(enfermedades[115:118])
for i,j in enumerate(medusaI[0]):
  for k,l in enumerate(j[1:]):
    medusaII.append(l)
medusamide=from_iterable(medusaII)
x9=list(set(medusamide))
diseases_by_molecule['medusamide']=x9

#papers about Credneramides related with diseases
credneI.append(enfermedades[118:125])
for i,j in enumerate(credneI[0]):
  for k,l in enumerate(j[1:]):
    credneII.append(l)
credneramides=from_iterable(credneII)
x10=list(set(credneramides))
diseases_by_molecule['credneramides']=x10

#papers about Seco-Briarellinone related with diseases
secoI.append(enfermedades[125:131])
for i,j in enumerate(secoI[0]):
  for k,l in enumerate(j[1:]):
    secoII.append(l)
seco_briarellinone=from_iterable(secoII)
x11=list(set(seco_briarellinone))
diseases_by_molecule['seco-briarellinone']=x11

#papers about Pseudopterane Diterpene related with diseases
pseudoI.append(enfermedades[131:150])
for i,j in enumerate(pseudoI[0]):
  for k,l in enumerate(j[1:]):
    pseudoII.append(l)
pseudopterane_diterpene=from_iterable(pseudoII)
x12=list(set(pseudopterane_diterpene))
diseases_by_molecule['pseudopterane diterpene']=x12

#papers about [d-MeAla11]-Epimer related with diseases
epimerI.append(enfermedades[150:])
for i,j in enumerate(epimerI[0]):
  for k,l in enumerate(j[1:]):
    epimerII.append(l)
epimer=from_iterable(epimerII)
x13=list(set(epimer))
diseases_by_molecule['[d-MeAla11]-epimer']=x13


#To create a dictionary with molecule:related diseases
print("In those papers, What diseases are these molecules related with?")
for k,v in diseases_by_molecule.items():
  print(k,' : ', v)  

In those papers, What diseases are these molecules related with?
coibanol  :  []
bastimolide  :  ['malaria', 'cancer', 'disease']
coibamide  :  ['malaria', 'cancer', 'disease']
briarellin  :  ['cancer', 'disease']
santacruzamate  :  ['cancer', 'disease']
gallinamide_A  :  []
coibacins  :  ['cancer', 'disease']
bacilosarcin  :  ['cancer', 'disease']
medusamide  :  ['cancer', 'disease']
credneramides  :  ['cancer', 'disease']
seco-briarellinone  :  ['cancer', 'disease']
pseudopterane diterpene  :  ['malaria', 'cancer', 'disease']
[d-MeAla11]-epimer  :  ['cancer']
